## Escenario pruebas 2 - Incluyendo nueva logica KM

In [1]:

import pandas as pd
import logging
import traceback
import json
from typing import Dict, List, Optional, Any
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from graph.perfil.state import (EstadoAnalisisPerfil, PerfilUsuario, InfoPasajeros, FiltrosInferidos, EconomiaUsuario, InfoClimaUsuario)
from utils.enums import (Transmision, NivelAventura, TipoUsoProfesional,  DimensionProblematica, EstiloConduccion, TipoMecanica, FrecuenciaUso, DistanciaTrayecto, FrecuenciaViajesLargos)

test_config = {"configurable": { "thread_id": "test_thread_notebook_1234"}}
from graph.perfil.nodes import (
    # calcular_recomendacion_economia_modo1_node, # Asumimos que existe pero no lo usaremos directamente si el modo es 2
   aplicar_filtros_pasajeros_node, aplicar_filtros_pasajeros_node,construir_filtros_node, calcular_flags_dinamicos_node,calcular_pesos_finales_node, formatear_tabla_resumen_node, calcular_km_anuales_postprocessing_node, buscar_coches_finales_node)

try:
    from config.llm import llm, llm_explicacion_coche
except ImportError:
    print("WARN: No se pudo importar la configuración de LLM. Las explicaciones podrían no funcionar.")
    llm_explicacion_coche = None

# Configurar logging básico para ver los DEBUG de los nodos
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)

# --- Configuración de Pandas para Mejor Visualización ---
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 150)


Entorno local detectado. Verificando GOOGLE_APPLICATION_CREDENTIALS del archivo .env...


In [2]:

PERFIL_A_PROBAR = "PERFIL" 
print(f"--- 🚗 CONFIGURANDO PRUEBA PARA PERFIL: {PERFIL_A_PROBAR} ---")

if PERFIL_A_PROBAR == "PERFIL":
    test_info_clima = InfoClimaUsuario(
        MUNICIPIO_ZBE=False, ZONA_LLUVIAS=False, ZONA_NIEBLAS=False, ZONA_NIEVE=False,
        ZONA_CLIMA_MONTA=False, ZONA_GLP=False, ZONA_GNV=False, cp_valido_encontrado=True
    )
    test_preferencias_usuario = PerfilUsuario(
        apasionado_motor="si", #activa premium y singular
        valora_estetica="si",  #activa estetica
        prefiere_diseno_exclusivo="si", #activa singular
        circula_principalmente_ciudad = 'no',
        uso_profesional="no", 
        tipo_uso_profesional=None, #TipoUsoProfesional.PASAJEROS/CARGA/MIXTO
        altura_mayor_190="no",
        coche_principal_hogar="si",
        rating_fiabilidad_durabilidad=9,
        rating_seguridad=5,
        rating_comodidad=5,
        rating_impacto_ambiental=5,
        rating_costes_uso=5, 
        rating_tecnologia_conectividad=5,
        transporta_carga_voluminosa="si", 
        necesita_espacio_objetos_especiales= "no",
        arrastra_remolque="no", 
        tiene_garage="si", 
        espacio_sobra_garage = "si",  #si/no 
        #problema_dimension_garage= [DimensionProblematica.LARGO , DimensionProblematica.ANCHO],
        problemas_aparcar_calle= "no", 
        tiene_punto_carga_propio="no",
        solo_electricos="no", 
        prioriza_baja_depreciacion="no",
        frecuencia_uso=FrecuenciaUso.FRECUENTEMENTE,
        distancia_trayecto=DistanciaTrayecto.ENTRE_10_Y_50_KM , 
        realiza_viajes_largos = 'si',
        frecuencia_viajes_largos= FrecuenciaViajesLargos.FRECUENTEMENTE, #FrecuenciaViajesLargos.OCASIONALMENTE,
        transmision_preferida=Transmision.AMBOS,
        estilo_conduccion=EstiloConduccion.DEPORTIVO,
        aventura=NivelAventura.ninguna,
        
    ) ##Si suele_llevar_acompanantes= false: info_pasajeros.frecuencia a "nunca".["nunca", "ocasional", "frecuente"]
    test_info_pasajeros = InfoPasajeros(suele_llevar_acompanantes= False ,
                                #frecuencia= None, 
                                frecuencia_viaje_con_acompanantes = None, #ocasional , frecuente
                                num_ninos_silla=0, num_otros_pasajeros=0) 
    
    
    test_economia_usuario = EconomiaUsuario(
        modo=2, submodo=1, pago_contado=30000.0
        )
    
    # test_economia_usuario = EconomiaUsuario(
    #  modo=1 submodo=None ingresos=50000.0 ahorro=25000.0 pago_contado=None 
    #  cuota_max=None entrada=None anos_posesion=8
    #     )


# 
initial_filtros_inferidos =  FiltrosInferidos( 
     #tipo_mecanica=[TipoMecanica.GASOLINA, TipoMecanica.DIESEL, TipoMecanica.GLP , TipoMecanica.GNV , TipoMecanica.HEVD , TipoMecanica.HEVG , TipoMecanica.PHEVG , TipoMecanica.PHEVD ] # Ejemplo
)

initial_state_dict: EstadoAnalisisPerfil = {
    "messages": [AIMessage(content="...")], 
    "info_clima_usuario": test_info_clima,
    "preferencias_usuario": test_preferencias_usuario, 
    "info_pasajeros": test_info_pasajeros,
    "economia": test_economia_usuario, 
    "filtros_inferidos": initial_filtros_inferidos,
    "pesos": None, "tabla_resumen_criterios": None, "coches_recomendados": None,
    "pregunta_pendiente": None, "penalizar_puertas_bajas": False, "priorizar_ancho": False,
    # Flags que se calcularán en calcular_flags_dinamicos_node
    "flag_penalizar_low_cost_comodidad": False, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_deportividad_comodidad": False, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_antiguo_por_tecnologia": False, # Lógica para Flag de Penalización por Antigüedad y Tecnología > 7
    "aplicar_logica_distintivo_ambiental": False,
    "es_municipio_zbe": False,
    "codigo_postal_extraido_temporal": None,
    "tipo_mensaje_cp_llm": None,
    "_decision_cp_validation": None,
    "penalizar_bev_reev_aventura_ocasional": False,
    "penalizar_phev_aventura_ocasional": False,
    "penalizar_electrificados_aventura_extrema": False,
    "favorecer_carroceria_montana": False,
    'favorecer_carroceria_comercial': False,
    'favorecer_carroceria_pasajeros_pro': False,
    'desfavorecer_carroceria_no_aventura': False, #Aventura 'ninguna' y no es clima de montaña. Activando flag para desfavorecer PICKUP/TODOTERRENO."
    'favorecer_suv_aventura_ocasional': False,
    "favorecer_pickup_todoterreno_aventura_extrema": False,
    "aplicar_logica_objetos_especiales": False,
    "favorecer_carroceria_confort": False,
    "flag_logica_uso_ocasional": False,
    "flag_favorecer_bev_uso_definido": False,
    "flag_penalizar_phev_uso_intensivo":False,
    "flag_favorecer_electrificados_por_punto_carga": False,  
    'penalizar_awd_ninguna_aventura' : False,
    'flag_bonus_awd_clima_adverso' : False,
    'favorecer_awd_aventura_ocasional' : False,
    'favorecer_awd_aventura_extrema' : False,
    'flag_logica_reductoras_aventura': False,
    'flag_logica_diesel_ciudad' : False,
    'flag_bonus_fiab_dur_critico': False,
    'flag_bonus_fiab_dur_fuerte': False,
    'flag_bonus_costes_critico': False
    
}
# --- 3. Ejecutar la Secuencia de Nodos de Finalización y Búsqueda ---
print("\n--- 🚀 INICIANDO SECUENCIA DE FINALIZACIÓN Y BÚSQUEDA ---")
current_state = initial_state_dict.copy()
try: 
    aplicar_filtros_pasajeros_node
    print("\n0. aplicar_filtros_pasajeros_node...")
    output_filtros = aplicar_filtros_pasajeros_node(current_state)
    current_state.update(output_filtros)
    
    print("\n0. Infiriendo filtros previos...")
    output_filtros = construir_filtros_node(current_state)
    current_state.update(output_filtros)
    #print(f"✅ filtros aplicado: {current_state.get('km_anuales_estimados')}")
       
    print("\n1. Calculando KM anuales...")
    output_km_anuales = calcular_km_anuales_postprocessing_node(current_state)
    current_state.update(output_km_anuales)
    print(f"✅ KM Anuales Estimados: {current_state.get('km_anuales_estimados')}")

    print("\n2. Calculando Flags dinámicos...")
    output_flags = calcular_flags_dinamicos_node(current_state)
    current_state.update(output_flags)

    print("\n3. Calculando Pesos finales...")
    output_pesos = calcular_pesos_finales_node(current_state)
    current_state.update(output_pesos)

    print("\n4. Generando Tabla Resumen de Criterios...")
    # Asumimos que tienes un nodo o función para esto
    # Si no, esta parte se puede comentar
    output_tabla = formatear_tabla_resumen_node(current_state)
    current_state.update(output_tabla)
    
    # --- ✅ 5. ANÁLISIS INTELIGENTE DEL RESULTADO (LA CORRECCIÓN) ---
    print("\n\n--- 🏁 ANÁLISIS DEL RESULTADO FINAL ---")
    
    print("\n5. 🚀 Ejecutando Búsqueda Final en BigQuery...")
    # La llamada final al nodo de búsqueda, usando tu nueva estructura
    output_final_busqueda = buscar_coches_finales_node(current_state, test_config)
    current_state.update(output_final_busqueda)
    
    # --- ✅ 5. ANÁLISIS INTELIGENTE DEL RESULTADO (LA CORRECCIÓN) ---
    print("\n\n--- 🏁 ANÁLISIS DEL RESULTADO FINAL ---")
    
    # Obtenemos el último mensaje generado por el agente
    final_message = current_state['messages'][-1] if current_state.get('messages') else None

    if isinstance(final_message, AIMessage):
        print("\n--- Mensaje de Texto (lo que ve el usuario) ---")
        # Usamos .pretty_print() para ver el texto formateado
        final_message.pretty_print()

        # Ahora, inspeccionamos el "bolsillo especial" para los datos estructurados
        if final_message.additional_kwargs and 'payload' in final_message.additional_kwargs:
            print("\n--- 🕵️ Datos Estructurados (lo que procesa el frontend) 🕵️ ---")
            payload = final_message.additional_kwargs['payload']
            
            # Verificamos si es una recomendación de coches
            if payload and payload.get('type') == 'car_recommendation':
                print(f"Tipo de Payload: {payload.get('type')}")
                
                coches_recomendados = payload.get('cars', [])
                if coches_recomendados:
                    print(f"\nSe encontraron {len(coches_recomendados)} coches en el payload:")
                    # Usamos Pandas para una visualización bonita de los datos
                    df = pd.DataFrame(coches_recomendados)
                    display(df)
                else:
                    print("⚠️ El payload no contenía coches.")
            else:
                print("⚠️ El payload no es del tipo 'car_recommendation' o está vacío.")
        else:
            # Esto se ejecutará si no se encontraron coches y el agente
            # devolvió un mensaje de texto simple.
            print("\n--- No se encontraron datos estructurados en additional_kwargs (¡Correcto si no hay coches!). ---")
    else:
        print("❌ No se generó ningún mensaje final del agente.")

except Exception as e_test:
    print(f"\n❌ ERROR durante la prueba de la secuencia de nodos: {e_test}")
    traceback.print_exc()
    
 

INFO:root:PostProc Filtros: Restricción geográfica -> Eliminando GLP.
INFO:root:PostProc Filtros: Restricción geográfica -> Eliminando GNV.
INFO:root:--- FIN DEBUG PostProc Filtros --- Filtros de mecánica finales construidos: [<TipoMecanica.BEV: 'BEV'>, <TipoMecanica.DIESEL: 'DIESEL'>, <TipoMecanica.FCEV: 'FCEV'>, <TipoMecanica.GASOLINA: 'GASOLINA'>, <TipoMecanica.HEVD: 'HEVD'>, <TipoMecanica.HEVG: 'HEVG'>, <TipoMecanica.MHEVD: 'MHEVD'>, <TipoMecanica.MHEVG: 'MHEVG'>, <TipoMecanica.PHEVD: 'PHEVD'>, <TipoMecanica.PHEVG: 'PHEVG'>, <TipoMecanica.REEV: 'REEV'>]
INFO:root:DEBUG (CalcKM) ► Cálculo: (52 * 4 * 30) + (300 * 60) = 24240 km/año
INFO:root:DEBUG (CalcFlags) ► Aventura 'Ninguna' y no es clima de montaña. Activando flag para desfavorecer PICKUP/TODOTERRENO.
INFO:root:DEBUG (CalcFlags) ► Nivel Aventura 'ninguna'. Activando PENALIZACIÓN para ALL.
INFO:root:Flags: Rating de Fiabilidad/Durabilidad CRÍTICO. Activando bonus.
INFO:root:Flags: Usuario no suele llevar acompañantes. Activando 

--- 🚗 CONFIGURANDO PRUEBA PARA PERFIL: PERFIL ---

--- 🚀 INICIANDO SECUENCIA DE FINALIZACIÓN Y BÚSQUEDA ---

0. aplicar_filtros_pasajeros_node...
--- Ejecutando Nodo: aplicar_filtros_pasajeros_node ---
DEBUG (Aplicar Filtros Pasajeros) ► Info recibida: freq='None', X=0, Z=0

0. Infiriendo filtros previos...
--- Ejecutando Nodo: inferir_filtros_node/construir_filtros_node ---
DEBUG (Filtros) ► Preferencias e info_clima disponibles. Construyendo filtros...
DEBUG (Filtros) ► Filtros finales construidos: tipo_mecanica=[<TipoMecanica.BEV: 'BEV'>, <TipoMecanica.DIESEL: 'DIESEL'>, <TipoMecanica.FCEV: 'FCEV'>, <TipoMecanica.GASOLINA: 'GASOLINA'>, <TipoMecanica.HEVD: 'HEVD'>, <TipoMecanica.HEVG: 'HEVG'>, <TipoMecanica.MHEVD: 'MHEVD'>, <TipoMecanica.MHEVG: 'MHEVG'>, <TipoMecanica.PHEVD: 'PHEVD'>, <TipoMecanica.PHEVG: 'PHEVG'>, <TipoMecanica.REEV: 'REEV'>] tipo_carroceria=None modo_adquisicion_recomendado=None precio_max_contado_recomendado=None cuota_max_calculada=None plazas_min=None

1. Calcul

INFO:root:✅ BigQuery query ejecutada, 12 resultados obtenidos.
INFO:root:INFO (Buscar BQ) ► 12 coches sanitizados y listos.
INFO:root:Log de búsqueda para conversación 'test_thread_notebook_1234' insertado en BigQuery.


INFO (BQ Logger) ► Log para 'test_thread_notebook_1234' guardado en BQ.


--- 🏁 ANÁLISIS DEL RESULTADO FINAL ---

--- Mensaje de Texto (lo que ve el usuario) ---
================================== Ai Message ==================================

¡Listo! Basado en todo lo que hablamos, aquí tienes 12 coche(s) que podrían interesarte:

--- 🕵️ Datos Estructurados (lo que procesa el frontend) 🕵️ ---
Tipo de Payload: car_recommendation

Se encontraron 12 coches en el payload:


,name,specs,imageUrl,price,score,analysis
0,1. Mercedes-Benz SLK 250,"[GASOLINA, 2012, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im02298-1-mercedes-slk-klasse.jpg,28.100€,41.09 pts,Análisis detallado de la recomendación pendiente de desarrollo.
1,2. BMW 640d Gran Coupé Steptronic,"[DIESEL, 2014, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im02695-1-bmw-6er-reihe-grancoupe.jpg,28.200€,38.88 pts,Análisis detallado de la recomendación pendiente de desarrollo.
2,3. BMW 430d Coupé Steptronic,"[DIESEL, 2015, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im02952-1-bmw-4er-reihe-coupe.jpg,28.000€,38.50 pts,Análisis detallado de la recomendación pendiente de desarrollo.
3,4. Mercedes-Benz CL 500 7G-TRONIC,"[GASOLINA, 2007, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im01658-1-mercedes-cl-klasse.jpg,21.900€,37.48 pts,Análisis detallado de la recomendación pendiente de desarrollo.
4,5. Porsche Boxster 2.7,"[GASOLINA, 2005, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im01067-1-porsche-boxster.jpg,26.990€,36.94 pts,Análisis detallado de la recomendación pendiente de desarrollo.
5,6. Mazda MX-5 RF SKYACTIV-G 131 Sports-Line,"[GASOLINA, 2019, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im04526-1-mazda-mx5.jpg,25.900€,36.66 pts,Análisis detallado de la recomendación pendiente de desarrollo.
6,7. Lexus NX 300h Business automatico,"[HEVG, 2019, FWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im05393-1-lexus-nx.jpg,27.990€,34.62 pts,Análisis detallado de la recomendación pendiente de desarrollo.
7,8. Hyundai Kona Elektro (64 kWh) Style,"[BEV, 2020, FWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im05548-1-hyundai-kona-e.jpg,19.290€,33.81 pts,Análisis detallado de la recomendación pendiente de desarrollo.
8,9. Audi Q2 1.4 TFSI cod design S tronic,"[GASOLINA, 2018, FWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im03874-1-audi-q2.jpg,18.950€,33.26 pts,Análisis detallado de la recomendación pendiente de desarrollo.
9,10. Audi A1 1.4 TFSI cod S tronic,"[GASOLINA, 2017, FWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im03219-1-audi-a1.jpg,21.800€,33.07 pts,Análisis detallado de la recomendación pendiente de desarrollo.


In [4]:

import pandas as pd
coches_recomendados = current_state[('coches_recomendados')]

df_resultados = pd.DataFrame(coches_recomendados)
        
#  OPCIÓN 1: Imprimir el DataFrame directamente (Pandas usará las opciones de display)
# print("\n--- Vista DataFrame (Pandas Display Options) ---")
# display(df_resultados) # 'display()' suele dar mejor formato en notebooks que 'print(df)'

#OPCIÓN 2: Convertir a Markdown (como en tu nodo, pero para consola)
print("\n--- Vista Markdown ---") 
columnas_deseadas = [
    # Columnas principales
    'nombre', 'ID', 'marca', 'modelo', 'tipo_mecanica', 'plazas' ,'km_ocasion'  ,'ano_unidad', 'anos_vehiculo', 'tipo_carroceria' ,'reductoras', 'traccion' ,'score_total', 'puntuacion_base', 'ajustes_experto',
    'dbg_score_estetica', 'estetica', 'dbg_score_premium', 'premium' ,'dbg_score_singular', 'singular', 'dbg_score_altura_libre', 'altura_libre_suelo',
    'dbg_score_batalla', 'batalla', 'dbg_score_altura_interior','indice_altura_interior',  'dbg_score_ancho', 'ancho' ,'dbg_score_largo','largo' ,'dbg_score_devaluacion', 'devaluacion',
    'dbg_score_maletero_min', 'maletero_minimo','dbg_score_maletero_max', 'maletero_maximo' ,
    'dbg_score_bajo_peso', 'peso' , 
    'dbg_score_par_remolque', 'par' ,
    'dbg_score_remolque_cf', 'capacidad_remolque_con_freno',
    'dbg_score_remolque_sf',  'capacidad_remolque_sin_freno',
    'dbg_score_menor_superficie', 'superficie_planta',
    'dbg_score_menor_giro', 'diametro_giro' , 
    'dbg_score_menor_largo', 'largo',
    'dbg_score_menor_ancho', 'ancho',
    'dbg_score_menor_alto', 'alto',
    'dbg_score_deportividad', 'deportividad',
    'dbg_score_menor_rel_peso_pot', 'relacion_peso_potencia',
    'dbg_score_potencia', 'potencia_maxima',
    'dbg_score_par_deportivo', 'par',
    'dbg_score_menor_aceleracion' ,'aceleracion_0_100',
    'dbg_score_autonomia_max', 'autonomia_uso_maxima', 
    'dbg_score_autonomia_2nd', 'autonomia_uso_2nd_drive',
    'dbg_score_menor_t_carga', 'tiempo_carga_min',
    'dbg_score_pot_carga_ac',  'potencia_maxima_carga_ac',
    'dbg_score_pot_carga_dc',  'potencia_maxima_carga_dc',
    'dbg_pen_km_extremo' ,
    'dbg_pen_puertas' , 'dbg_pen_low_cost_comodidad' , 'dbg_pen_deportividad_comodidad' ,
    'dbg_pen_antiguedad' , 'dbg_ajuste_distintivo' , 'dbg_bonus_ocasion_ambiental' , 'dbg_ajuste_zbe' ,
    'dbg_pen_bev_reev_avent_ocas' , 'dbg_pen_phev_avent_ocas' , 'dbg_pen_electrif_avent_extr' ,
    'dbg_bonus_car_montana' , 'dbg_bonus_car_comercial' , 'dbg_bonus_car_pasajeros' , 'dbg_pen_car_no_aventura' ,
    'dbg_bonus_suv_avent_ocas' , 'dbg_bonus_tt_avent_extr' , 'dbg_bonus_pickup_avent_extr' , 
    'dbg_ajuste_objetos_especiales' , 'dbg_bonus_car_confort' , 'dbg_bonus_ocasion_uso_ocas' ,
    'dbg_pen_electrif_uso_ocas' , 'dbg_bonus_bev_uso_definido' , 'dbg_pen_phev_uso_intensivo' ,
    'dbg_bonus_punto_carga' , 'dbg_ajuste_awd_aventura' , 'dbg_bonus_awd_nieve' , 'dbg_bonus_awd_montana' ,
    'dbg_bonus_reductoras' , 'dbg_ajuste_diesel_ciudad' , 'dbg_ajuste_km_anuales', 
    'dbg_bonus_seguridad', 'seguridad', 
    'dbg_bonus_fiabilidad' ,'fiabilidad', 
    'dbg_bonus_durabilidad' , 'durabilidad', 
    'indice_consumo_energia','dbg_bonus_bajo_consumo', 'costes_de_uso'  , 'dbg_bonus_coste_uso' ,  
    'dbg_bonus_coste_mantenimiento' ,'costes_mantenimiento' ,'dbg_pen_antiguedad_general','dbg_pen_tamano_no_compacto', 
    'dbg_bonus_lifestyle' ,  'dbg_ajuste_deportividad_lifestyle' ,'dbg_ajuste_maletero_personal' , 'dbg_bonus_coche_ciudad' , 'dbg_bonus_coche_ciudad_2'
]
columnas_existentes = [col for col in columnas_deseadas if col in df_resultados.columns]
if columnas_existentes:
            if 'score_total' in df_resultados.columns: # Formatear score para legibilidad
                df_resultados['score_total'] = df_resultados['score_total'].apply(lambda x: f"{x:.4f}" if isinstance(x, float) else x)
            print(df_resultados[columnas_existentes].to_markdown(index=False))
else:
            print("WARN: No se encontraron columnas esperadas para mostrar en formato tabla.")
            print("Resultados crudos (lista de dicts):", coches_recomendados) 


--- Vista Markdown ---
| nombre                                   |     ID | marca         | modelo      | tipo_mecanica   |   plazas |   km_ocasion |   ano_unidad |   anos_vehiculo | tipo_carroceria   | reductoras   | traccion   |   score_total |   puntuacion_base |   ajustes_experto |   dbg_score_estetica |   estetica |   dbg_score_premium |   premium |   dbg_score_singular |   singular |   dbg_score_altura_libre |   altura_libre_suelo |   dbg_score_batalla |   batalla |   dbg_score_altura_interior |   indice_altura_interior |   dbg_score_ancho |   ancho |   dbg_score_largo |   largo |   dbg_score_devaluacion |   devaluacion |   dbg_score_maletero_min |   maletero_minimo |   dbg_score_maletero_max |   maletero_maximo |   dbg_score_bajo_peso |   peso |   dbg_score_par_remolque |   par |   dbg_score_remolque_cf |   capacidad_remolque_con_freno |   dbg_score_remolque_sf |   capacidad_remolque_sin_freno |   dbg_score_menor_superficie |   superficie_planta |   dbg_score_menor_giro |   di

In [3]:
print('flag_penalizar_tamano_no_compacto:' , current_state['flag_penalizar_tamano_no_compacto'])
print('-----------------------') 
print('flag_bonus_singularidad_lifestyle:' , current_state['flag_bonus_singularidad_lifestyle']) 
print('-----------------------') 
print('flag_deportividad_lifestyle:' , current_state['flag_deportividad_lifestyle'])
print('-----------------------') 
print('flag_ajuste_maletero_personal:' , current_state['flag_ajuste_maletero_personal'])
print('-----------------------') 
print('flag_coche_ciudad_perfil:' , current_state['flag_coche_ciudad_perfil'])
print('-----------------------') 
print('flag_coche_ciudad_2_perfil:' , current_state['flag_coche_ciudad_2_perfil'])
print('-----------------------') 
print('flag_ajuste_maletero_personal:' , current_state['flag_ajuste_maletero_personal'])

flag_penalizar_tamano_no_compacto: True
-----------------------
flag_bonus_singularidad_lifestyle: True
-----------------------
flag_deportividad_lifestyle: True
-----------------------
flag_ajuste_maletero_personal: True
-----------------------
flag_coche_ciudad_perfil: False
-----------------------
flag_coche_ciudad_2_perfil: False
-----------------------
flag_ajuste_maletero_personal: True


In [5]:
print(type(current_state['flag_bonus_fiab_dur_critico']))


<class 'bool'>


In [ ]:
print(current_state.keys())

In [6]:
print(current_state['codigo_postal_usuario'])
print('-----------------------')
print(current_state['info_clima_usuario'])
print('-----------------------')
print(current_state['preferencias_usuario'])
print('-----------------------')
print(current_state['filtros_inferidos'])
print('-----------------------')
print(current_state['info_pasajeros'])
print('-----------------------')
print(current_state['economia'])
print('-----------------------')
print(current_state['pesos'])
print('-----------------------')
print("info_pasajeros:",current_state['info_pasajeros'])
print('-----------------------')

None
-----------------------
MUNICIPIO_ZBE=False ZONA_LLUVIAS=False ZONA_NIEBLAS=False ZONA_NIEVE=False ZONA_CLIMA_MONTA=False ZONA_GLP=False ZONA_GNV=False cp_valido_encontrado=True codigo_postal_consultado=None
-----------------------
apasionado_motor='no' valora_estetica='no' coche_principal_hogar='si' frecuencia_uso=<FrecuenciaUso.FRECUENTEMENTE: 'frecuentemente'> distancia_trayecto=<DistanciaTrayecto.ENTRE_10_Y_50_KM: 'está entre 10 y 50 km'> realiza_viajes_largos='si' frecuencia_viajes_largos=<FrecuenciaViajesLargos.FRECUENTEMENTE: 'frecuentemente'> circula_principalmente_ciudad='no' uso_profesional='no' tipo_uso_profesional=None prefiere_diseno_exclusivo='si' altura_mayor_190='no' transporta_carga_voluminosa='si' necesita_espacio_objetos_especiales='no' arrastra_remolque='no' tiene_garage='si' problemas_aparcar_calle='no' espacio_sobra_garage='si' problema_dimension_garage=None tiene_punto_carga_propio='no' aventura=<NivelAventura.ninguna: 'ninguna'> estilo_conduccion=<EstiloCon

In [7]:
print("flag_penalizar_deportividad_comodidad:", current_state['flag_penalizar_deportividad_comodidad'])
print('-----------------------')
print("flag_penalizar_low_cost_comodidad:", current_state['flag_penalizar_low_cost_comodidad'])
print('-----------------------')
print("penalizar_puertas_bajas:" , current_state['penalizar_puertas_bajas'])
print('-----------------------')
print("flag_penalizar_antiguo_por_tecnologia:" , current_state['flag_penalizar_antiguo_por_tecnologia'])
print('-----------------------')
print("es municipio_zbe: " , current_state['es_municipio_zbe'])
print('-----------------------')
print("penalizar_electrificados_aventura_extrema: " , current_state['penalizar_electrificados_aventura_extrema'])
print('-----------------------')
print('favorecer_carroceria_montana:', current_state['favorecer_carroceria_montana'])
print('-----------------------')
print('favorecer_carroceria_comercial', current_state['favorecer_carroceria_comercial'])
print('-----------------------')
print('favorecer_carroceria_pasajeros_pro:', current_state['favorecer_carroceria_pasajeros_pro'])
print('-----------------------')
print('aplicar_logica_objetos_especiales:', current_state['aplicar_logica_objetos_especiales'])#OK 
print('-----------------------')
print('favorecer_carroceria_confort:', current_state['favorecer_carroceria_confort']) #OK
print('-----------------------')
print('flag_logica_uso_ocasional:' , current_state['flag_logica_uso_ocasional'])
print('-----------------------')
print('flag_favorecer_bev_uso_definido:' , current_state['flag_favorecer_bev_uso_definido'])
print('-----------------------')
print('flag_penalizar_phev_uso_intensivo:' , current_state['flag_penalizar_phev_uso_intensivo'])
print('-----------------------')
print('flag_favorecer_electrificados_por_punto_carga:' , current_state['flag_favorecer_electrificados_por_punto_carga'])
print('-----------------------')
print('km_anuales_estimados:' , current_state['km_anuales_estimados'])

flag_penalizar_deportividad_comodidad: False
-----------------------
flag_penalizar_low_cost_comodidad: False
-----------------------
penalizar_puertas_bajas: False
-----------------------
flag_penalizar_antiguo_por_tecnologia: False
-----------------------
es municipio_zbe:  False
-----------------------
penalizar_electrificados_aventura_extrema:  False
-----------------------
favorecer_carroceria_montana: False
-----------------------
favorecer_carroceria_comercial False
-----------------------
favorecer_carroceria_pasajeros_pro: False
-----------------------
aplicar_logica_objetos_especiales: False
-----------------------
favorecer_carroceria_confort: False
-----------------------
flag_logica_uso_ocasional: False
-----------------------
flag_favorecer_bev_uso_definido: False
-----------------------
flag_penalizar_phev_uso_intensivo: False
-----------------------
flag_favorecer_electrificados_por_punto_carga: False
-----------------------
km_anuales_estimados: 24240


In [ ]:

print('desfavorecer_carroceria_no_aventura:', current_state['desfavorecer_carroceria_no_aventura']) #Aventura nula y no en montaña desfavorece PICKUP/TODOTERRENO
print('-----------------------')
print('favorecer_suv_aventura_ocasional:', current_state['favorecer_suv_aventura_ocasional']) #OK
print('-----------------------')
print('favorecer_pickup_todoterreno_aventura_extrema:', current_state['favorecer_pickup_todoterreno_aventura_extrema'])
print('-----------------------')
print("penalizar_bev_reev_aventura_ocasional: " , current_state['penalizar_bev_reev_aventura_ocasional'])
print('-----------------------')
print("penalizar_phev_aventura_ocasional: " , current_state['penalizar_phev_aventura_ocasional'])
print('-----------------------')
print('penalizar_awd_ninguna_aventura:' , current_state['penalizar_awd_ninguna_aventura'])
print('-----------------------')
print('favorecer_awd_aventura_ocasional:' , current_state['favorecer_awd_aventura_ocasional'])
print('-----------------------')
print('favorecer_awd_aventura_extrema:' , current_state['favorecer_awd_aventura_extrema'])
print('-----------------------')
print('flag_bonus_awd_nieve:' , current_state['flag_bonus_awd_nieve'])
print('-----------------------')
print('flag_bonus_awd_montana:' , current_state['flag_bonus_awd_montana'])
print('-----------------------')
print('flag_logica_reductoras_aventura:' , current_state['flag_logica_reductoras_aventura'])
print('-----------------------')
print('flag_logica_diesel_ciudad:' , current_state['flag_logica_diesel_ciudad'])
print('-----------------------') 
print('flag_bonus_awd_clima_adverso:' , current_state['flag_bonus_awd_clima_adverso']) 
print('-----------------------') 
print('flag_bonus_seguridad_critico:' , current_state['flag_bonus_seguridad_critico'])
print('-----------------------') 
print('flag_bonus_seguridad_fuerte:' , current_state['flag_bonus_seguridad_fuerte'])
print('-----------------------')
print("aplicar_logica_distintivo_ambiental:" ,  current_state['aplicar_logica_distintivo_ambiental'])
print('-----------------------') 
print('flag_bonus_fiab_dur_critico' , current_state['flag_bonus_fiab_dur_critico'])
print('-----------------------') 
print('flag_bonus_fiab_dur_fuerte:' , current_state['flag_bonus_fiab_dur_fuerte'])
print('-----------------------')
print('flag_bonus_costes_critico') , current_state['flag_bonus_costes_critico']

In [ ]:
print(current_state['coches_recomendados']('columnas_deseadas'))

In [ ]:
df_resultados[columnas_existentes].to_csv(
    "resultados_filtrados.csv",
    index=False,
    sep=";",
    float_format="%.4f",
    encoding="utf-8"
)


# df_resultados.to_csv(
#     "resultados_filtradosV2.csv",
#     index=False,
#     sep=";",
#     float_format="%.4f",
#     encoding="utf-8"
# )
